## Sveska za importovanje i pravljenje jobova za CAS-VIT mrežu

CAS-ViT se zasniva na Visual Transformer arhitekturi, više informacija na linku.

https://paperswithcode.com/paper/cas-vit-convolutional-additive-self-attention

(samo sam sortirao po broju parametara i uzeo ovu random mrezu, nemam pojma da li je dobra, i rad za sada ne razumem)

Ova sveska uz prateće fajlove automatski skida weightove za CAS-ViT model zadate veličine, i posle taj model šalje na AIHUB za testiranje.

Za sada sam pokrenuo samo xs model, koji se izvršava super brzo i ima manje parametara od MobileNetV2, doduše mnogo više layera koji su raspoređeni na više COMPUTE unita, što je koliko ja razumem dobro.

Ne znam da li je dobro da modele čuvamo na GIT-u (Drakula neka kaže kako bi to trebalo), pa sam sve fajlove koje se generišu stavio u gitignore.


In [18]:
import gdown
import torch
import rcvit
import torchviz
import torchsummary
import numpy as np
import qai_hub as hub
import requests
import input_getter
import make_jobs
import skimage as ski
import helper
import tfhelper

In [19]:
import os

# Model selection (options: "xs", "s", "m", "t")

modelID = "s"

driveFileIDs = dict()

driveFileIDs["xs"] = "16wKcwF6QMW5w_lyPYnDKjMNuoxQDfrLK"
driveFileIDs["s"]  = "1facFRq8s8oelYUtK1fj3fcfdoWoKDBQQ"
driveFileIDs["m"]  = "13sQpSEf0h_uuh0jRy9V0yIW6ZsbDpVGy"
driveFileIDs["t"]  = "1NqoIUPbwBC91RTjTUvubAbOfGqo1VYT0"

networks = dict()
networks["xs"] = rcvit.rcvit_xs
networks["s"]  = rcvit.rcvit_s
networks["m"]  = rcvit.rcvit_m
networks["t"]  = rcvit.rcvit_t

file_id = driveFileIDs[modelID]

modelName = "CASVIT_{}".format(modelID)

file_path = modelName + ".pth"

if not os.path.exists(file_path):
    gdown.download(f"https://drive.google.com/uc?id={file_id}", file_path, quiet=False)
    print("Model downloaded successfully.")
else:
    print(f"File {file_path} already exists.")

File CASVIT_s.pth already exists.


In [20]:
net: rcvit.RCViT = networks[modelID]()
print(net)

RCViT(
  (patch_embed): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (network): ModuleList(
    (0): Sequential(
      (0): AdditiveBlock(
        (local_perception): LocalIntegration(
          (network): Sequential(
            (0): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1))
            (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48)
            (3): GELU(approximate='none')
            (4): Conv2d(48, 48, kernel_size=(1, 1), stride=(1, 1))
          )
        )
        (norm1): BatchNorm2d(48, eps=1e-05, momentum=0.1, aff

In [21]:
torchsummary.summary(net, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 24, 112, 112]             672
       BatchNorm2d-2         [-1, 24, 112, 112]              48
              ReLU-3         [-1, 24, 112, 112]               0
            Conv2d-4           [-1, 48, 56, 56]          10,416
       BatchNorm2d-5           [-1, 48, 56, 56]              96
              ReLU-6           [-1, 48, 56, 56]               0
            Conv2d-7           [-1, 48, 56, 56]           2,352
       BatchNorm2d-8           [-1, 48, 56, 56]              96
            Conv2d-9           [-1, 48, 56, 56]             480
             GELU-10           [-1, 48, 56, 56]               0
           Conv2d-11           [-1, 48, 56, 56]           2,352
 LocalIntegration-12           [-1, 48, 56, 56]               0
      BatchNorm2d-13           [-1, 48, 56, 56]              96
           Conv2d-14          [-1, 144,

In [22]:
# Loading pre-trained weights

weightDict = torch.load(file_path, map_location=torch.device("cpu"))
net.load_state_dict(weightDict["model"])

C:\Users\Pro\AppData\Local\Temp\ipykernel_26120\588262210.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weightDict = torch.load(file_path, map_location=torch.device("c

<All keys matched successfully>

In [23]:
net.eval()
input = input_getter.local_image_getter('../data/1.jpeg')
net(torch.tensor(input.get_input_torch())).shape

C:\Users\Pro\AppData\Local\Temp\ipykernel_26120\3979637909.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  net(torch.tensor(input.get_input_torch())).shape


torch.Size([1, 1000])

In [24]:
input.get_input_numpy().shape

(1, 3, 224, 224)

In [25]:
type(input.get_input_numpy())

numpy.ndarray

In [26]:
# Kada se uradi ovi jobovi na linkovima se moze naci vizuelizacija mreze i informacije za dalju analizu

compile, profile, inference = make_jobs.compile_profile_inference(net, input)

Uploading tmp8ay0m8yk.pt


100%|██████████| 23.6M/23.6M [00:03<00:00, 7.18MB/s]


Scheduled compile job (jpyzz7rrg) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jpyzz7rrg/

Waiting for compile job (jpyzz7rrg) completion. Type Ctrl+C to stop waiting at any time.
    ✅ SUCCESS                          
Scheduled profile job (jp044v32g) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jp044v32g/



Uploading dataset: 533kB [00:00, 584kB/s]                            4.51MB/s]


Scheduled inference job (jp82240zp) successfully. To see the status and results:
    https://app.aihub.qualcomm.com/jobs/jp82240zp/



In [27]:
categories = helper.get_imagenet_categories()

# Rezultati dobijeni na cloudu

helper.inference_job_probabilities(inference)

Waiting for inference job (jp82240zp) completion. Type Ctrl+C to stop waiting at any time.
    ✅ SUCCESS                          


tmphh5y__8d.h5: 100%|██████████| 14.5k/14.5k [00:00<00:00, 14.8MB/s]


Top-5 predictions for Cloud model on :
472 b'canoe'              72.4%
975 b'lakeside'           19.6%
449 b'boathouse'           2.8%
693 b'paddle'              1.5%
536 b'dock'                0.5%


In [28]:
# Vraca verovatnoce lokalne PyTorch mreze

helper.print_probablities_from_output(net(input.get_input_torch()), categories=categories, modelname = 'Torch Local')

Top-5 predictions for Torch Local on :
472 b'canoe'              83.1%
975 b'lakeside'           11.1%
449 b'boathouse'           1.5%
693 b'paddle'              1.4%
536 b'dock'                0.2%


In [29]:
# Vraca verovatnoce TensorFlow mreze

download_path = tfhelper.download_model_from_compile_job(compile, modelName)
TFModel = tfhelper.TFHelper(download_path)
TFModel.run_inference(input.get_input_numpy())
helper.print_probablities_from_output(TFModel.run_inference(input.get_input_numpy()), categories=categories, modelname = 'TensorFlow')

CASVIT_s.tflite: 100%|██████████| 22.1M/22.1M [00:01<00:00, 14.5MB/s]

Downloaded model to CASVIT_s.tflite
Top-5 predictions for TensorFlow on :
472 b'canoe'              83.1%
975 b'lakeside'           11.1%
449 b'boathouse'           1.5%
693 b'paddle'              1.4%
536 b'dock'                0.2%
